In [1]:
import pandas as pd
#행렬 연산 패키지
import numpy as np
#데이터 시각화 패키지
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns

%matplotlib inline

#Scaler
from sklearn.preprocessing import StandardScaler
#data split
from sklearn.model_selection import train_test_split#graphviz: 나무 구조 시각화(.dot 확장자 파일 불러오기 등)
from sklearn.tree import export_graphviz
import graphviz

#주성분 분석
from sklearn.decomposition import PCA
import statsmodels.api as sm


#회귀 분석
from sklearn.linear_model import LinearRegression
#후진제거법
from sklearn.feature_selection import RFE

#예측
#예측/회귀 Decision tree
from sklearn.tree import DecisionTreeRegressor
#예측/회귀 Random Forest
from sklearn.ensemble import RandomForestRegressor
#예측/회귀 Gradient
from sklearn.ensemble import GradientBoostingRegressor
#최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#ARIMA
from statsmodels.tsa.arima_model import ARIMA
#회귀분석
from statsmodels.formula.api import ols
#선형모델 formula(y ~ x1 + x2 + ...)
import statsmodels.formula.api as smf
#다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import qqplot, add_constant
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score#분류
#로지스틱 회귀
from statsmodels.api import Logit
#의사결정나무
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
#그래디언트부스트
from sklearn.ensemble import GradientBoostingClassifier
#엑스지부스트
from xgboost import XGBClassifier
from xgboost import plot_importance
#SVM
from sklearn.svm import SVC
#분류 NN (MLP)
from sklearn.neural_network import MLPClassifier
#KNN 분류
from sklearn.neighbors import KNeighborsClassifier#분류모델 평가 함수
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc#사용할 함수 선언#RSME: root mean squared error 함수
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))#MAPE: mean absolutre percentage error 함수
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)/ y_true))*100#XG Boost에서 사용할 정확도 표현 함수
def eval_class_model(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precesion = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬 \n',confusion, '\n')
    print('정확도: ', accuracy)
    print('정밀도: ', precesion)
    print('재현율: ', recall)
    print('F1: ', F1)
    print('AUC: ', AUC)#for ignoring warning
import warnings
warnings.filterwarnings('ignore')#음수표시
[f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name]
matplotlib.rc("axes", unicode_minus = False)
plt.rcParams["font.family"] = 'NanumGothic'

# 데이터 읽기

In [3]:
df_raw = pd.read_csv('tele_customer_final.csv')
df_raw = df_raw.drop(labels = 'Unnamed: 0',axis = 1)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerId            6859 non-null   object 
 1   Gender                6859 non-null   object 
 2   Age                   6859 non-null   int64  
 3   AgeGroup              6859 non-null   object 
 4   Married               6859 non-null   object 
 5   Dependents            6859 non-null   object 
 6   noDependents          6859 non-null   int64  
 7   Referrals             6859 non-null   object 
 8   noReferrals           6859 non-null   int64  
 9   PaperlessBilling      6859 non-null   object 
 10  PaymentMethod         6859 non-null   object 
 11  OnlineSecurity        6859 non-null   object 
 12  OnlineBackup          6859 non-null   object 
 13  TechSupport           6859 non-null   object 
 14  UnlimitedData         6859 non-null   object 
 15  AvgDownloadGB        

# 새롭게 산출된 Data

최종 데이터

In [4]:
df_raw_choose = df_raw[['Monthly_charge','AvgDownloadGB','noReferrals','TechSupport','OnlineSecurity','ChurnLabel']]
df_raw_choose['ChurnLabel'] = df_raw['ChurnLabel'].replace({'Yes': 1, 'No': 0})

In [5]:
#get dummy
df_raw_choose_dummy = pd.get_dummies(df_raw_choose)

## 최종 DT, RF, GB에 쓸 데이터 

In [6]:
df_raw_choose_dummy

,Monthly_charge,AvgDownloadGB,noReferrals,ChurnLabel,TechSupport_No,TechSupport_Yes,OnlineSecurity_No,OnlineSecurity_Yes
0,83590.476190,16,2,0,0,1,1,0
1,68928.571429,10,0,0,1,0,1,0
2,98180.000000,30,0,1,1,0,1,0
3,112742.857143,4,1,1,1,0,1,0
4,94500.000000,11,3,1,0,1,1,0
...,...,...,...,...,...,...,...,...
6854,64428.000000,59,0,0,0,1,0,1
6855,101891.304348,17,1,1,1,0,1,0
6856,56600.000000,51,0,0,1,0,1,0
6857,80107.594937,58,5,0,0,1,0,1


## data split

In [7]:
#더미화한 변수를 train set과 test set으로 분리한다
df_raw_choose_x = df_raw_choose_dummy.drop(["ChurnLabel"], axis = 1, inplace = False) #목표변수 외
df_raw_choose_y = df_raw_choose_dummy["ChurnLabel"] #목표변수

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_choose_x, df_raw_choose_y, test_size = 0.4, random_state = 42)
df_train, df_test = train_test_split(df_raw_choose_dummy, test_size = 0.4, random_state = 42)
print("분할 전 설명변수 데이터 :", df_raw_choose_x.shape)
print("분할 후 설명변수 데이터 :", df_train_x.shape, "\n Test", df_test_x.shape)

# 선형회귀 돌리고 나온 데이터

분할 전 설명변수 데이터 : (6859, 7)
분할 후 설명변수 데이터 : (4115, 7) 
 Test (2744, 7)


# 분류

# 의사결정나무

In [8]:
dt_uncustomized = RandomForestClassifier(random_state = 42)
dt_uncustomized.fit(df_train_x, df_train_y)
# Train 데이터 설명력
print("Score on training set: {:.3f}".format(dt_uncustomized.score(df_train_x, df_train_y)))
# Test 데이터 설명력
print("Score on test set: {:.3f}".format(dt_uncustomized.score(df_test_x, df_test_y)))

Score on training set: 0.998
Score on test set: 0.747


In [9]:
#train 및 설명력 결과 저장
train_score = []
test_score = []

#param 지정을 위한 범위
para_leaf = [n_leaf * 1 for n_leaf in range(1, 20)]
para_split = [n_split * 2 for n_split in range(2, 20)]
para_depth = [depth for depth in range(1, 11)]


## Grid SearchCV

estimator = DecisionTreeClassifier(random_state = 42)
#구하고자 하는 parameter와 범위
param_grid = {"max_depth": para_depth, "min_samples_split": para_split, "min_samples_leaf": para_leaf}
#설명력이 높은 최적 parameter 찾기
grid_dt = GridSearchCV(estimator, param_grid, scoring = "r2", n_jobs = -1)
grid_dt.fit(df_train_x, df_train_y)

print("best estimator model:\n{}".format(grid_dt.best_estimator_))
print("\nbest parameter:\n{}".format(grid_dt.best_params_))
print("\nbest score:\n{}".format(grid_dt.best_score_.round(3)))

In [10]:
# GridSearch를 이용하여 최적 모델 결정
tree_final = DecisionTreeClassifier(random_state = 42, \
                                   max_depth = 7, \
                                   min_samples_leaf = 16, \
                                   min_samples_split = 4)
tree_final.fit(df_train_x, df_train_y)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=16, min_samples_split=4,
                       random_state=42)

# 이탈하지 않는 고객 중 이탈할 고객 예측하기

In [11]:
df_raw_chosen = df_raw_choose[df_raw_choose['ChurnLabel'] == 0]

In [12]:
df_raw_chosen_dummy = pd.get_dummies(df_raw_chosen)

In [13]:
#더미화한 변수를 train set과 test set으로 분리한다
df_raw_chosen_x = df_raw_chosen_dummy.drop(["ChurnLabel"], axis = 1, inplace = False) #목표변수 외
df_raw_chosen_y = df_raw_chosen_dummy["ChurnLabel"] #목표변수

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_chosen_x, df_raw_chosen_y, test_size = 0.4, random_state = 42)
df_train, df_test = train_test_split(df_raw_chosen_dummy, test_size = 0.4, random_state = 42)
print("분할 전 설명변수 데이터 :", df_raw_chosen_x.shape)
print("분할 후 설명변수 데이터 :", df_train_x.shape, "\n Test", df_test_x.shape)


분할 전 설명변수 데이터 : (5158, 7)
분할 후 설명변수 데이터 : (3094, 7) 
 Test (2064, 7)


## 이탈 여부를 확인을 위한 y_predict

In [14]:
#최종 의사결정나무 모델을 이용하여 이탈하지 않는 고객 중 이탈할 고객 예측하기
y_predict = tree_final.predict(df_raw_chosen_x)

In [15]:
#실제 고객 데이터 생성해서 예측
real_cust = pd.DataFrame()
real_cust['Monthly_charge'] = [78000]
real_cust['AvgDownloadGB'] = [70]
real_cust['noReferrals'] = [0]
real_cust['TechSupport_No'] = [1]
real_cust['TechSupport_Yes'] = [0]
real_cust['OnlineSecurity_No'] = [1]
real_cust['OnlineSecurity_Yes'] = [0]
tree_final.predict(real_cust)

array([1], dtype=int64)

In [16]:
#예측한 결과
for i in y_predict:
    print(i)

0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
1
0
0
0
0
1
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
1
0
0


`y_predict = 1`일 때, 이탈

In [17]:
#총 탈하지 않는 고객 수 구하기
len(y_predict)

5158

In [18]:
#df_raw_chosen dataFrame에 Y_predict열 추가
df_raw_chosen['Y_predict'] = np.NaN
for i in range(len(df_raw_chosen)):
    df_raw_chosen['Y_predict'][i] = y_predict[i]

In [19]:
#이탈할 고객 df_raw_chosen_danger 로 저장
df_raw_chosen_danger = df_raw_chosen[df_raw_chosen['Y_predict']==1]

In [20]:
df_raw_chosen_danger.to_csv('df_raw_chosen_danger_no_index.csv')

In [21]:
df_raw_chosen_danger = df_raw_chosen_danger.iloc[:,1:]

In [22]:
df_raw_chosen_danger

,AvgDownloadGB,noReferrals,TechSupport,OnlineSecurity,ChurnLabel,Y_predict
9,14,3,Yes,Yes,0,1.0
14,19,0,Yes,Yes,0,1.0
16,20,0,Yes,Yes,0,1.0
22,30,1,No,No,0,1.0
43,21,0,Yes,Yes,0,1.0
...,...,...,...,...,...,...
5079,7,0,No,No,0,1.0
5082,11,0,No,No,0,1.0
5110,41,1,No,Yes,0,1.0
5131,19,0,No,No,0,1.0


In [23]:
df_raw_chosen_danger.to_csv('df_raw_chosen_danger.csv')

In [24]:
tree_final.fit(df_train_x, df_train_y)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=16, min_samples_split=4,
                       random_state=42)

In [25]:
df_train_x

,Monthly_charge,AvgDownloadGB,noReferrals,TechSupport_No,TechSupport_Yes,OnlineSecurity_No,OnlineSecurity_Yes
4359,23139.130435,0,0,1,0,1,0
3774,119660.975610,19,10,0,1,0,1
5298,47170.833333,71,5,0,1,0,1
6123,62414.285714,59,0,1,0,1,0
5717,135302.380952,21,2,0,1,0,1
...,...,...,...,...,...,...,...
5897,29409.210526,0,0,1,0,1,0
635,82131.578947,42,0,1,0,1,0
4152,29531.707317,0,7,1,0,1,0
5042,22956.944444,0,2,1,0,1,0


In [26]:
df_train_y

4359    0
3774    0
5298    0
6123    0
5717    0
       ..
5897    0
635     0
4152    0
5042    0
1152    0
Name: ChurnLabel, Length: 3094, dtype: int64

In [28]:
#train data 설명력
print("Score on training set: {:.3f}".format(tree_final.score(df_train_x, df_train_y)))
#test data 설명력
print("Score on test set: {:.3f}".format(tree_final.score(df_test_x, df_test_y)))

Score on training set: 1.000
Score on test set: 1.000
